# DEEPFACE EMBEDDING AND SAVE

## Move correct folder

In [14]:
%cd data_generation

/mnt/d/deepface-video-recognition/data_generation


## IMPORT LIB

In [1]:
from deepface import DeepFace

import concurrent.futures
import os
import pickle
import numpy as np
import faiss
import queue
import threading
import cv2
import matplotlib.pyplot as plt

2024-09-17 12:09:32.442540: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 12:09:32.442613: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 12:09:32.445442: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 12:09:34.937302: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Initial Parameters and Function

In [8]:
fixed_dim = 2622
workers = 5
movies = ['Calloused_Hands', 'Liberty_Kid', 'like_me', 'losing_ground', 'Memphis']
movie = "Liberty_Kid"
# detect_faces_folder = "det_faces"
character_query_folder = "character_query"
character_emb_query_folder = "character_emb_query"
faces_folder = "faces"
faces_emb_folder = "faces_emb"
emb_folder_mapping = {
    "ArcFace": "ArcFace",
    "VGG-Face": "VggFace",
    "Facenet": "FaceNet",
    "GhostFaceNet": "GhostFaceNet"
}

embs = ["ArcFace", "VGG-Face", "Facenet", "GhostFaceNet"]

## Embedding Function

In [3]:
def create_file_path(source_file_path, source_folder, destination_folder):
    # Construct destination file path
    destination_file_path = os.path.join(destination_folder, os.path.relpath(source_file_path, source_folder))

    destination_dir = os.path.dirname(destination_file_path)
    # Create destination folder if not exists
    os.makedirs(destination_dir, exist_ok=True)
    return destination_file_path

In [4]:
def pad_embedding(embedding, target_dim):
    """Pad or truncate embedding to the target dimension."""
    if len(embedding) >= target_dim:
        return embedding[:target_dim]
    else:
        padding = np.zeros(target_dim - len(embedding))
        return np.concatenate((embedding, padding))

In [5]:
def attach_model_to_path(file_name, model):
    if "det_opencv" in file_name:
        new_filename = file_name.replace("det_opencv", f"det_opencv-emb_{model}")
    elif "det_fastmtcnn" in file_name:
        new_filename = file_name.replace("det_fastmtcnn", f"det_fastmtcnn-emb_{model}")
    elif "det_retinaface" in file_name:
        new_filename = file_name.replace("det_retinaface", f"det_retinaface-emb_{model}")
    else:
        new_filename = file_name
    return new_filename

In [6]:
def extract_emb_parallel(source_file_path, model_emb_file_path, model):
    global fixed_dim
    embeddings = DeepFace.represent(img_path = source_file_path,
                                        model_name = model,
                                       enforce_detection = False)
    original_embedding = np.array(embeddings[0]['embedding'], dtype=np.float32)
    padded_embedding = pad_embedding(original_embedding, fixed_dim)
    padded_embedding = np.expand_dims(padded_embedding, axis=0)  # Add batch dimension
    with open(model_emb_file_path, "wb") as f:
        pickle.dump(padded_embedding, f)
        print(f"Created embeddings: {source_file_path} -> {model_emb_file_path}")

In [7]:
def extract_emb(source_folder: str, des_em_folder: str, model: str):
    # Traverse to all files and subfolder in source folder
    for root, _, files in os.walk(source_folder):
        with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
            for file in files:
                # Construct source file path
                source_file_path = os.path.join(root, file)
                emb_file_path = create_file_path(source_file_path, source_folder, des_em_folder)
                model_emb_file_path = attach_model_to_path(os.path.splitext(emb_file_path)[0] +  ".pkl", emb_folder_mapping[model])
                if source_file_path.lower().endswith('.jpg') and os.path.exists(model_emb_file_path) == False:
                    executor.submit(extract_emb_parallel, source_file_path, model_emb_file_path, model)
                    # extract_emb_parallel(source_file_path, model_emb_file_path, model)
                else:
                    print(f"{source_file_path} not end with jpg or {model_emb_file_path} exists")

## Create character faces embedding

In [9]:
def create_option_faces_emb(model):
    global movie, character_query_folder, character_emb_query_folder
    folder = emb_folder_mapping[model]
    source_det_faces = f'../{character_query_folder}/{movie}/'
    des_em_char = f"../{character_emb_query_folder}/{movie}/{folder}/"
    extract_emb(source_det_faces, des_em_char, model)

In [15]:
create_option_faces_emb("ArcFace")

../character_query/Liberty_Kid/Derrick/Derrick_1.jpg not end with jpg or ../character_emb_query/Liberty_Kid/ArcFace/Derrick/Derrick_1.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_2.jpg not end with jpg or ../character_emb_query/Liberty_Kid/ArcFace/Derrick/Derrick_2.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_3.jpg not end with jpg or ../character_emb_query/Liberty_Kid/ArcFace/Derrick/Derrick_3.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_4.jpg not end with jpg or ../character_emb_query/Liberty_Kid/ArcFace/Derrick/Derrick_4.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_5.jpg not end with jpg or ../character_emb_query/Liberty_Kid/ArcFace/Derrick/Derrick_5.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_6.jpg not end with jpg or ../character_emb_query/Liberty_Kid/ArcFace/Derrick/Derrick_6.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_7.jpg not end with jpg or ../character_emb_query/Liberty_Kid/ArcFace/Derrick/Derric

In [16]:
create_option_faces_emb("VGG-Face")

2024-09-17 12:11:50.383463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-17 12:11:50.789031: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-17 12:11:50.789461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-17 12:11:50.797734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-17 12:11:50.798109: I tensorflow/compile

Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_2.jpg -> ../character_emb_query/Liberty_Kid/VggFace/Derrick/Derrick_2.pkl
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_4.jpg -> ../character_emb_query/Liberty_Kid/VggFace/Derrick/Derrick_4.pkl
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_1.jpg -> ../character_emb_query/Liberty_Kid/VggFace/Derrick/Derrick_1.pkl
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_5.jpg -> ../character_emb_query/Liberty_Kid/VggFace/Derrick/Derrick_5.pkl
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_3.jpg -> ../character_emb_query/Liberty_Kid/VggFace/Derrick/Derrick_3.pkl
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_6.jpg -> ../character_emb_query/Liberty_Kid/VggFace/Derrick/Derrick_6.pkl
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_7.jpg -> ../character_emb_query/Liberty_Kid/VggFace/Derrick/Derrick_7.pkl


In [17]:
create_option_faces_emb("Facenet")

../character_query/Liberty_Kid/Derrick/Derrick_1.jpg not end with jpg or ../character_emb_query/Liberty_Kid/FaceNet/Derrick/Derrick_1.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_2.jpg not end with jpg or ../character_emb_query/Liberty_Kid/FaceNet/Derrick/Derrick_2.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_3.jpg not end with jpg or ../character_emb_query/Liberty_Kid/FaceNet/Derrick/Derrick_3.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_4.jpg not end with jpg or ../character_emb_query/Liberty_Kid/FaceNet/Derrick/Derrick_4.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_5.jpg not end with jpg or ../character_emb_query/Liberty_Kid/FaceNet/Derrick/Derrick_5.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_6.jpg not end with jpg or ../character_emb_query/Liberty_Kid/FaceNet/Derrick/Derrick_6.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_7.jpg not end with jpg or ../character_emb_query/Liberty_Kid/FaceNet/Derrick/Derric

In [18]:
create_option_faces_emb("GhostFaceNet")

../character_query/Liberty_Kid/Derrick/Derrick_1.jpg not end with jpg or ../character_emb_query/Liberty_Kid/GhostFaceNet/Derrick/Derrick_1.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_2.jpg not end with jpg or ../character_emb_query/Liberty_Kid/GhostFaceNet/Derrick/Derrick_2.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_3.jpg not end with jpg or ../character_emb_query/Liberty_Kid/GhostFaceNet/Derrick/Derrick_3.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_6.jpg not end with jpg or ../character_emb_query/Liberty_Kid/GhostFaceNet/Derrick/Derrick_6.pkl exists
../character_query/Liberty_Kid/Derrick/Derrick_7.jpg not end with jpg or ../character_emb_query/Liberty_Kid/GhostFaceNet/Derrick/Derrick_7.pkl exists
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_5.jpg -> ../character_emb_query/Liberty_Kid/GhostFaceNet/Derrick/Derrick_5.pkl
Created embeddings: ../character_query/Liberty_Kid/Derrick/Derrick_4.jpg -> ../character_emb_query/Liber

## Create all faces embedding

In [10]:
def create_faces_emb(model):
    DeepFace.represent(img_path = "test.jpg",
                                    model_name = model,
                                    enforce_detection = False)
    global movie, faces_emb_folder, faces_folder
    source_faces= f"../{faces_folder}/{movie}"
    des_emb_faces = f"../{faces_emb_folder}/{movie}"
    extract_emb(source_faces, des_emb_faces, model)

In [ ]:
# create_faces_emb("Facenet")

In [ ]:
# create_faces_emb("VGG-Face")

In [ ]:
# create_faces_emb("ArcFace")

In [ ]:
# create_faces_emb("GhostFaceNet")

In [ ]:
# print("Done")